# Data Splits Truncated

Here, we split in the same way as in `data_splits.ipynb`, but then we restrict the number of points in the training data.
There are two ways we could do this:
1. Take the original splits and drop random examples. Problem: As we go to smaller training sets, this will lead to us inadvertently removing all the examples of one compound from the training data but not from validation/test.
2. We do (procedurally) the same splits as before, but with different train-test ratios.

Option 2 is safer to do w.r.t. retaining the split dimensionality.

### 0D Split
For the 0D split, we use a random train-test split.
Standard: We use a 80/10/10 split into train, val, and test set.

Truncated: 
- 40/30/30 -> 16k training samples
- 20/40/40 -> 8k training samples
- 10/45/45 -> 4k training samples
- 5/50/45 -> 2k training samples
- 2.5/50/47.5 -> 1k training samples
- 1.25/50/48.75 -> 500 training samples
(numbers of training samples are averaged over folds and approximate)

### 1D Split
For the 1D split, we use a (1D) GroupShuffleSplit.
Standard: Each individual split will be 80/10/10 train/test (of groups not samples!).
As groups, we use either initiator, monomer, or terminator (3x3 splits).

Truncated:
(for uniform results, we pick only one dimension to split on: Initiators)
- 80/10/10 -> 31,250 training samples (this is not equivalent to the "standard" 1D split b/c here we split on initiators 9 times)
- 40/30/30 -> 15,600 training samples
- 20/40/40 -> 8,100 training samples
- 10/45/45 -> 3,680 training samples
- 5/50/45 -> 1,668 training samples
- 2.5/50/47.5 -> 487 training samples
- 1.25/50/48.75 -> not possible b/c training set will be empty on some folds
(numbers of training samples are averaged over folds and approximate)


### other splits
For the 2D and 3D split, we don't perform this rather expensive experiment

In [1]:
import pathlib
import sys

sys.path.append(str(pathlib.Path().resolve().parents[1]))

import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, ShuffleSplit

from src.definitions import DATA_DIR
from src.util.train_test_split import GroupShuffleSplitND

In [2]:
# Load data
data_filename = "synferm_dataset_2023-09-05_40018records.csv"
data_name = data_filename.rsplit("_", maxsplit=1)[0]
df = pd.read_csv(DATA_DIR / "curated_data" / data_filename)
df.shape

(40018, 27)

In [3]:
df.head()

,I_long,M_long,T_long,product_A_smiles,I_smiles,M_smiles,T_smiles,reaction_smiles,reaction_smiles_atom_mapped,experiment_id,...,binary_H,scaled_A,scaled_B,scaled_C,scaled_D,scaled_E,scaled_F,scaled_G,scaled_H,major_A-C
0,2-Pyr003,Fused002,TerABT004,COc1ccc(CCOC(=O)N2C[C@H](NC(=O)c3cccc(Cl)n3)[C...,O=C(c1cccc(Cl)n1)[B-](F)(F)F.[K+],COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)O...,Nc1ccc(F)cc1S,O=C(c1cccc(Cl)n1)[B-](F)(F)F.COc1ccc(CCOC(=O)N...,F[B-](F)(F)[C:2]([c:1]1[cH:16][cH:18][cH:20][c...,56113,...,0,0.036021,0.003427,0.0,0.020975,0.002958,0.941981,0.914281,0.00000,A
1,2-Pyr003,Fused002,TerABT007,COc1ccc(CCOC(=O)N2C[C@H](NC(=O)c3cccc(Cl)n3)[C...,O=C(c1cccc(Cl)n1)[B-](F)(F)F.[K+],COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)O...,Nc1cc(Br)ccc1S,O=C(c1cccc(Cl)n1)[B-](F)(F)F.COc1ccc(CCOC(=O)N...,F[B-](F)(F)[C:2]([c:1]1[cH:16][cH:18][cH:20][c...,56114,...,0,0.000000,0.000000,0.0,0.006159,0.364398,0.928851,1.106548,0.00000,no_product
2,2-Pyr003,Fused002,TerABT013,COc1ccc(CCOC(=O)N2C[C@H](NC(=O)c3cccc(Cl)n3)[C...,O=C(c1cccc(Cl)n1)[B-](F)(F)F.[K+],COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)O...,Nc1cc(C(F)(F)F)ccc1S,O=C(c1cccc(Cl)n1)[B-](F)(F)F.COc1ccc(CCOC(=O)N...,F[B-](F)(F)[C:2]([c:1]1[cH:16][cH:18][cH:20][c...,56106,...,1,0.000000,0.000000,0.0,0.014212,2.166420,1.013596,0.537785,0.05686,no_product
3,2-Pyr003,Fused002,TerABT014,COc1ccc(CCOC(=O)N2C[C@H](NC(=O)c3cccc(Cl)n3)[C...,O=C(c1cccc(Cl)n1)[B-](F)(F)F.[K+],COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)O...,Nc1ccc(Cl)cc1S,O=C(c1cccc(Cl)n1)[B-](F)(F)F.COc1ccc(CCOC(=O)N...,F[B-](F)(F)[C:2]([c:1]1[cH:16][cH:18][cH:20][c...,56112,...,0,0.028915,0.005039,0.0,0.015578,0.504057,0.992614,0.890646,0.00000,A
4,2-Pyr003,Fused002,TerTH001,COc1ccc(CCOC(=O)N2C[C@H](NC(=O)c3cccc(Cl)n3)[C...,O=C(c1cccc(Cl)n1)[B-](F)(F)F.[K+],COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)O...,[Cl-].[NH3+]NC(=S)c1ccccc1,O=C(c1cccc(Cl)n1)[B-](F)(F)F.COc1ccc(CCOC(=O)N...,F[B-](F)(F)[C:2]([c:1]1[cH:13][cH:15][cH:17][c...,56109,...,0,0.350061,0.643219,0.0,0.031689,0.613596,0.109309,0.439018,0.00000,B


In [4]:
#define a write function that we can reuse later
def write_indices_and_stats(indices, sizes, pos_class_A, split_dimension, train_size, save_indices=True):
    """
    Write function that can be reused for the other splits
    Args:
        indices: list of 3-tuples
        sizes: list of 3-tuples, length equal to indices
        pos_class_A: list of 3-tuples, length equal to indices
        split_dimension: str or int, e.g. "0", "1", "2", "3"
        save_indices: bool, whether to save the indices. Useful if we need to regenerate statistics. Default: True
        train_size: ratio of train samples from all samples. Give as percentage"
    """
    n_folds = len(indices)
    save_dir = DATA_DIR / "curated_data" / "splits" / f"{data_name}_{split_dimension}D_split_{train_size}"
    save_dir.mkdir(parents=True, exist_ok=True)
    if save_indices:
        for i, (idx_train, idx_val, idx_test) in enumerate(indices):
            with open(save_dir / f"fold{i}_train.csv", "w") as f:
                f.write("index\n")
                f.write("\n".join([str(i) for i in idx_train]))
                
            with open(save_dir / f"fold{i}_val.csv", "w") as f:
                f.write("index\n")
                f.write("\n".join([str(i) for i in idx_val]))
                
            with open(save_dir / f"fold{i}_test.csv", "w") as f:
                f.write("index\n")
                f.write("\n".join([str(i) for i in idx_test]))
    
    for i, (size, count_A) in enumerate(zip(sizes, pos_class_A)):
        with open(save_dir / f"fold{i}_statistics.txt", "w") as f:
            f.write(f"Train samples: {size[0]} ({size[0]/len(df):.1%})\n")
            f.write(f"Val samples: {size[1]} ({size[1]/len(df):.1%})\n")
            f.write(f"Test samples: {size[2]} ({size[2]/len(df):.1%})\n")
            if split_dimension > 1:
                f.write(f"Not used: {len(df) - np.sum(size):.0f} ({(len(df) - np.sum(size)) / len(df):.1%})\n")
            f.write(f"Train samples binary_A has label 1: {count_A[0]} ({count_A[0]/size[0]:.1%})\n")
            f.write(f"Val samples binary_A has label 1: {count_A[1]} ({count_A[1]/size[1]:.1%})\n")
            f.write(f"Test samples binary_A has label 1: {count_A[2]} ({count_A[2]/size[2]:.1%})\n")
            
    # summary statistics
    sum_pos_class_A = np.sum(pos_class_A, axis=0)
    sum_sizes = np.sum(sizes, axis=0)
    with open(save_dir / "summary_statistics.txt", "w") as f:
        f.write(f"Mean Train samples: {sum_sizes[0] / n_folds:.0f} ({sum_sizes[0] / n_folds / len(df):.1%})\n")
        f.write(f"Mean Val samples: {sum_sizes[1] / n_folds:.0f} ({sum_sizes[1] / n_folds / len(df):.1%})\n")
        f.write(f"Mean Test samples: {sum_sizes[2] / n_folds:.0f} ({sum_sizes[2] / n_folds / len(df):.1%})\n")
        if split_dimension > 1:
            f.write(f"Not used: {len(df) - np.sum(sum_sizes) / n_folds:.0f} ({(len(df) - np.sum(sum_sizes) / n_folds) / len(df):.1%})\n")
        f.write(f"Mean Train samples binary_A has label 1: {sum_pos_class_A[0] / n_folds:.0f} ({sum_pos_class_A[0]/sum_sizes[0]:.1%})\n")
        f.write(f"Mean Val samples binary_A has label 1: {sum_pos_class_A[1] / n_folds:.0f} ({sum_pos_class_A[1]/sum_sizes[1]:.1%})\n")
        f.write(f"Mean Test samples binary_A has label 1: {sum_pos_class_A[2] / n_folds:.0f} ({sum_pos_class_A[2]/sum_sizes[2]:.1%})\n")

## 0D split

In [5]:
splitter = ShuffleSplit(n_splits=9, test_size=0.3, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.3/0.7, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))

print(sizes)
print(pos_class_A)

In [7]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=0, save_indices=True, train_size=40)

In [8]:
splitter = ShuffleSplit(n_splits=9, test_size=0.4, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.4/0.6, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))

print(sizes)
print(pos_class_A)

[(8003, 16007, 16008), (8003, 16007, 16008), (8003, 16007, 16008), (8003, 16007, 16008), (8003, 16007, 16008), (8003, 16007, 16008), (8003, 16007, 16008), (8003, 16007, 16008), (8003, 16007, 16008)]
[(6550, 12944, 13038), (6466, 13069, 12997), (6551, 12973, 13008), (6501, 13031, 13000), (6498, 13019, 13015), (6529, 12949, 13054), (6487, 12956, 13089), (6511, 12983, 13038), (6470, 12997, 13065)]


In [9]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=0, save_indices=True, train_size=20)

In [10]:
splitter = ShuffleSplit(n_splits=9, test_size=0.45, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.45/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))

print(sizes)
print(pos_class_A)

[(4001, 18008, 18009), (4001, 18008, 18009), (4001, 18008, 18009), (4001, 18008, 18009), (4001, 18008, 18009), (4001, 18008, 18009), (4001, 18008, 18009), (4001, 18008, 18009), (4001, 18008, 18009)]
[(3258, 14585, 14689), (3258, 14636, 14638), (3262, 14628, 14642), (3285, 14622, 14625), (3272, 14611, 14649), (3248, 14584, 14700), (3208, 14597, 14727), (3266, 14622, 14644), (3264, 14572, 14696)]


In [11]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=0, save_indices=True, train_size=10)

In [12]:
splitter = ShuffleSplit(n_splits=9, test_size=0.45, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.5/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))

print(sizes)
print(pos_class_A)

[(2000, 20009, 18009), (2000, 20009, 18009), (2000, 20009, 18009), (2000, 20009, 18009), (2000, 20009, 18009), (2000, 20009, 18009), (2000, 20009, 18009), (2000, 20009, 18009), (2000, 20009, 18009)]
[(1639, 16204, 14689), (1626, 16268, 14638), (1644, 16246, 14642), (1639, 16268, 14625), (1642, 16241, 14649), (1632, 16200, 14700), (1600, 16205, 14727), (1615, 16273, 14644), (1630, 16206, 14696)]


In [13]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=0, save_indices=True, train_size=5)

In [14]:
splitter = ShuffleSplit(n_splits=9, test_size=0.475, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.5/0.525, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))

print(sizes)
print(pos_class_A)

[(1000, 20009, 19009), (1000, 20009, 19009), (1000, 20009, 19009), (1000, 20009, 19009), (1000, 20009, 19009), (1000, 20009, 19009), (1000, 20009, 19009), (1000, 20009, 19009), (1000, 20009, 19009)]
[(830, 16213, 15489), (810, 16280, 15442), (803, 16283, 15446), (819, 16298, 15415), (813, 16271, 15448), (828, 16174, 15530), (825, 16155, 15552), (814, 16268, 15450), (838, 16198, 15496)]


In [15]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=0, save_indices=True, train_size=2.5)

In [16]:
splitter = ShuffleSplit(n_splits=9, test_size=0.4875, random_state=42)
inner_splitter = ShuffleSplit(n_splits=1, test_size=0.5/0.5125, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(df):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))

print(sizes)
print(pos_class_A)

[(500, 20009, 19509), (500, 20009, 19509), (500, 20009, 19509), (500, 20009, 19509), (500, 20009, 19509), (500, 20009, 19509), (500, 20009, 19509), (500, 20009, 19509), (500, 20009, 19509)]
[(425, 16221, 15886), (409, 16283, 15840), (399, 16286, 15847), (421, 16315, 15796), (416, 16251, 15865), (408, 16193, 15931), (394, 16192, 15946), (401, 16269, 15862), (408, 16223, 15901)]


In [17]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=0, save_indices=True, train_size=1.25)

## 1D split

In [5]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.1, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.1/0.9, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))
print(sizes)
print(pos_class_A)

[(30974, 4318, 4726), (32233, 3546, 4239), (32488, 4020, 3510), (29520, 6094, 4404), (31196, 4704, 4118), (32186, 4686, 3146), (30353, 4855, 4810), (30367, 5326, 4325), (31930, 4193, 3895)]
[(25183, 3332, 4017), (26439, 2886, 3207), (26725, 3018, 2789), (23667, 5073, 3792), (25283, 3811, 3438), (26321, 4005, 2206), (24318, 4093, 4121), (24334, 4641, 3557), (25753, 3495, 3284)]


In [6]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=1, save_indices=True, train_size=80)

In [7]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.3, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.3/0.7, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))
print(sizes)
print(pos_class_A)

[(17126, 10733, 12159), (16875, 11529, 11614), (15464, 12107, 12447), (15857, 11436, 12725), (15219, 12660, 12139), (15958, 12613, 11447), (14167, 11735, 14116), (16400, 11582, 12036), (13764, 13270, 12984)]
[(13955, 8318, 10259), (13586, 9676, 9270), (12577, 9769, 10186), (13126, 9179, 10227), (12787, 10139, 9606), (13432, 10341, 8759), (11156, 9398, 11978), (13012, 9529, 9991), (10817, 11257, 10458)]


In [9]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=1, save_indices=True, train_size=40)

In [10]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.4, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.4/0.6, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))
print(sizes)
print(pos_class_A)

[(8818, 16152, 15048), (9070, 14595, 16353), (6805, 17281, 15932), (8445, 15367, 16206), (8844, 15312, 15862), (8004, 15372, 16642), (7254, 15824, 16940), (7631, 15875, 16512), (8090, 15386, 16542)]
[(7327, 13007, 12198), (7765, 11387, 13380), (5521, 14416, 12595), (7227, 12271, 13034), (7406, 12375, 12751), (6713, 12612, 13207), (5823, 12409, 14300), (6262, 12499, 13771), (6508, 12661, 13363)]


In [11]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=1, save_indices=True, train_size=20)

In [12]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.45, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.45/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))
print(sizes)
print(pos_class_A)

[(3567, 18651, 17800), (4020, 17884, 18114), (4219, 17970, 17829), (3452, 17898, 18668), (4571, 16516, 18931), (3014, 17745, 19259), (2607, 18079, 19332), (3703, 16819, 19496), (3970, 17106, 18942)]
[(2839, 15154, 14539), (3381, 14306, 14845), (3556, 14845, 14131), (2843, 14698, 14991), (3966, 13229, 15337), (2433, 14651, 15448), (2127, 14232, 16173), (3037, 13186, 16309), (3279, 13820, 15433)]


In [13]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=1, save_indices=True, train_size=10)

In [17]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.45, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.5/0.55, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))
print(sizes)
print(pos_class_A)

[(1447, 20771, 17800), (2409, 19495, 18114), (1278, 20911, 17829), (1983, 19367, 18668), (1643, 19444, 18931), (1721, 19038, 19259), (1240, 19446, 19332), (1243, 19279, 19496), (2046, 19030, 18942)]
[(1081, 16912, 14539), (2068, 15619, 14845), (1023, 17378, 14131), (1696, 15845, 14991), (1359, 15836, 15337), (1448, 15636, 15448), (1015, 15344, 16173), (980, 15243, 16309), (1710, 15389, 15433)]


In [18]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=1, save_indices=True, train_size=5)

In [19]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.475, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.5/0.525, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))
print(sizes)
print(pos_class_A)

[(202, 21461, 18355), (947, 20029, 19042), (483, 21397, 18138), (609, 19716, 19693), (626, 19940, 19452), (472, 19775, 19771), (202, 20239, 19577), (484, 19470, 20064), (359, 20234, 19425)]
[(147, 17398, 14987), (821, 16126, 15585), (382, 17770, 14380), (537, 16114, 15881), (501, 16225, 15806), (381, 16344, 15807), (147, 16026, 16359), (387, 15325, 16820), (289, 16428, 15815)]


In [20]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=1, save_indices=True, train_size=2.5)

In [21]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.4875, random_state=42)
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.5/0.5125, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

indices = []
sizes = []
pos_class_A = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    # inner split
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class_A.append((np.sum(df['binary_A'][idx_train]), np.sum(df['binary_A'][idx_val]), np.sum(df['binary_A'][idx_test])))
print(sizes)
print(pos_class_A)

ValueError: With n_samples=34, test_size=0.9756097560975611 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [9]:
write_indices_and_stats(indices, sizes, pos_class_A, split_dimension=1, save_indices=True, train_size=40)